In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import Input, add, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import threading
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.data_utils import Sequence

from losses import (
    binary_crossentropy,
    dice_loss,
    bce_dice_loss,
    dice_coef,
    weighted_bce_dice_loss
)



## Model

In [2]:

def encoder(x, filters=44, n_block=3, kernel_size=(3, 3), activation='relu'):
    skip = []
    for i in range(n_block):
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        skip.append(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    return x, skip


def bottleneck(x, filters_bottleneck, mode='cascade', depth=6,
               kernel_size=(3, 3), activation='relu'):
    dilated_layers = []
    if mode == 'cascade':  # used in the competition
        for i in range(depth):
            x = Conv2D(filters_bottleneck, kernel_size,
                       activation=activation, padding='same', dilation_rate=2**i)(x)
            dilated_layers.append(x)
        return add(dilated_layers)
    elif mode == 'parallel':  # Like "Atrous Spatial Pyramid Pooling"
        for i in range(depth):
            dilated_layers.append(
                Conv2D(filters_bottleneck, kernel_size,
                       activation=activation, padding='same', dilation_rate=2**i)(x)
            )
        return add(dilated_layers)


def decoder(x, skip, filters, n_block=3, kernel_size=(3, 3), activation='relu'):
    for i in reversed(range(n_block)):
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        x = concatenate([skip[i], x])
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
        x = Conv2D(filters * 2**i, kernel_size, activation=activation, padding='same')(x)
    return x


def get_dilated_unet(
        input_shape=(1920, 1280, 3),
        mode='cascade',
        filters=44,
        n_block=3,
        lr=0.0001,
        loss=bce_dice_loss,
        n_class=1
):
    inputs = Input(input_shape)
    
    enc, skip = encoder(inputs, filters, n_block)
    bottle = bottleneck(enc, filters_bottleneck=filters * 2**n_block, mode=mode)
    dec = decoder(bottle, skip, filters, n_block)
    classify = Conv2D(n_class, (1, 1), activation='sigmoid')(dec)

    model = Model(inputs=inputs, outputs=classify)
    model.compile(optimizer=RMSprop(lr), loss=loss, metrics=[dice_coef])

    return model

# Data

In [14]:
WIDTH = 1280
HEIGHT = 720
BATCH_SIZE = 2

def train_generator(id_list):
    while True:
        shuffle_indices = np.arange(len(id_list))
        shuffle_indices = np.random.permutation(shuffle_indices)
        
        for start in range(0, len(id_list), BATCH_SIZE):
            x_batch = []
            y_batch = []
            
            end = min(start + BATCH_SIZE, len(id_list))
            ids_batch = id_list[shuffle_indices[start:end]]
            
            for _id in ids_batch:
                img = cv2.imread('input/images/{}'.format(_id))
                mask = cv2.imread('input/masks/A_alpha_{}.png'.format(_id.replace('.jpg', '')), cv2.IMREAD_GRAYSCALE)
                mask = np.expand_dims(mask, axis=-1)
                assert mask.ndim == 3
                
                # === You can add data augmentations here. === #
                if np.random.random() < 0.5:
                    img, mask = img[:, ::-1, :], mask[..., ::-1, :]  # random horizontal flip
                
                x_batch.append(img)
                y_batch.append(mask)
            
            x_batch = np.array(x_batch, np.float32) / 255.
            y_batch = np.array(y_batch, np.float32) / 255.
            
            yield x_batch, y_batch


def valid_generator(id_list):
    while True:
        for start in range(0, len(id_list), BATCH_SIZE):
            x_batch = []
            y_batch = []

            end = min(start + BATCH_SIZE, len(id_list))
            ids_batch = id_list[start:end]
            for _id in ids_batch:
                img = cv2.imread('input/images/{}'.format(_id))
                mask = cv2.imread('input/masks/A_alpha_{}.png'.format(_id.replace('.jpg', '')), cv2.IMREAD_GRAYSCALE)
                mask = np.expand_dims(mask, axis=-1)
                
                assert mask.ndim == 3
    
                x_batch.append(img)
                y_batch.append(mask)
            
            x_batch = np.array(x_batch, np.float32) / 255.
            y_batch = np.array(y_batch, np.float32) / 255.
            
            yield x_batch, y_batch




In [15]:
Training

In [18]:
with open('train_ids.txt') as f:
    all_ids = [line.rstrip() for line in f.readlines()]

ids_train, ids_valid = train_test_split(all_ids, test_size=0.1, random_state=42)
ids_train, ids_valid = np.array(ids_train), np.array(ids_valid)

model = get_dilated_unet(
    input_shape=(HEIGHT, WIDTH, 3),
    mode='cascade',
    filters=32,
    n_class=1
)

callbacks = [EarlyStopping(monitor='val_dice_coef',
                           patience=10,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_coef',
                               factor=0.2,
                               patience=5,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_coef',
                             filepath='model_weights.hdf5',
                             save_best_only=True,
                             mode='max')]

model.fit_generator(generator=train_generator(ids_train),
                    steps_per_epoch=np.ceil(float(len(ids_train)) / float(BATCH_SIZE)),
                    epochs=100,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(ids_valid),
                    validation_steps=np.ceil(float(len(ids_valid)) / float(BATCH_SIZE)))

W1112 11:48:38.189123 140515320526592 callbacks.py:1791] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


Epoch 1/100
405/405 [==============================] - 248s 612ms/step - loss: 1.0566 - dice_coef: 0.4205 - val_loss: 0.8472 - val_dice_coef: 0.6367
Epoch 2/100
405/405 [==============================] - 242s 597ms/step - loss: 0.8393 - dice_coef: 0.6415 - val_loss: 0.7331 - val_dice_coef: 0.7059
Epoch 3/100
405/405 [==============================] - 242s 597ms/step - loss: 0.7915 - dice_coef: 0.6659 - val_loss: 0.6594 - val_dice_coef: 0.7211
Epoch 4/100
405/405 [==============================] - 242s 597ms/step - loss: 0.7438 - dice_coef: 0.6937 - val_loss: 0.6828 - val_dice_coef: 0.7308
Epoch 5/100
405/405 [==============================] - 242s 597ms/step - loss: 0.6924 - dice_coef: 0.7227 - val_loss: 0.6211 - val_dice_coef: 0.7470
Epoch 6/100
405/405 [==============================] - 242s 597ms/step - loss: 0.6621 - dice_coef: 0.7438 - val_loss: 0.6715 - val_dice_coef: 0.7560
Epoch 7/100
405/405 [==============================] - 242s 597ms/step - loss: 0.6104 - dice_coef: 0.7664 

405/405 [==============================] - 245s 606ms/step - loss: 0.0484 - dice_coef: 0.9770 - val_loss: 0.3323 - val_dice_coef: 0.9271
Epoch 54/100
405/405 [==============================] - 246s 606ms/step - loss: 0.0462 - dice_coef: 0.9781 - val_loss: 0.3390 - val_dice_coef: 0.9265
Epoch 55/100
405/405 [==============================] - 246s 606ms/step - loss: 0.0448 - dice_coef: 0.9783 - val_loss: 0.3890 - val_dice_coef: 0.9242
Epoch 56/100
405/405 [==============================] - 245s 606ms/step - loss: 0.0469 - dice_coef: 0.9774 - val_loss: 0.3814 - val_dice_coef: 0.9252
Epoch 57/100
405/405 [==============================] - 246s 607ms/step - loss: 0.0438 - dice_coef: 0.9785 - val_loss: 0.3421 - val_dice_coef: 0.9282
Epoch 58/100
405/405 [==============================] - 246s 606ms/step - loss: 0.0471 - dice_coef: 0.9782 - val_loss: 0.3556 - val_dice_coef: 0.9267
Epoch 59/100
405/405 [==============================] - 246s 606ms/step - loss: 0.0430 - dice_coef: 0.9788 - val_

KeyboardInterrupt: 

In [48]:
from matplotlib import pyplot as plt


for _id in ids_valid:
    print(_id)
    img = cv2.imread('input/images/{}'.format(_id))
    mask = cv2.imread('input/masks/A_alpha_{}.png'.format(_id.replace('.jpg', '')), cv2.IMREAD_GRAYSCALE)
    mask = np.expand_dims(mask, axis=-1)

    img = np.array([img], np.float32) / 255.
    mask = np.array([mask], np.float32) / 255.
    preds = model.predict(img)
    clean_pred = np.array((preds[0,:,:,0] > 0.5)*255)
    cv2.imwrite('output/{}.png'.format(_id.replace('.jpg', '')), clean_pred)
    


0479_0479.jpg
0091_0091.jpg
0765_0765.jpg
0253_0253.jpg
0044_0044.jpg
0766_0766.jpg
0024_0024.jpg
0141_0141.jpg
0990_0990.jpg
0365_0365.jpg
0003_0003.jpg
0892_0892.jpg
0862_0862.jpg
0730_0730.jpg
0170_0170.jpg
0964_0964.jpg
0381_0381.jpg
0340_0340.jpg
0866_0866.jpg
0314_0314.jpg
0582_0582.jpg
0725_0725.jpg
0472_0472.jpg
0278_0278.jpg
0037_0037.jpg
0583_0583.jpg
0850_0850.jpg
0387_0387.jpg
0441_0441.jpg
0745_0745.jpg
0581_0581.jpg
0881_0881.jpg
0861_0861.jpg
0480_0480.jpg
0186_0186.jpg
0899_0899.jpg
0237_0237.jpg
0303_0303.jpg
0693_0693.jpg
0595_0595.jpg
0593_0593.jpg
0477_0477.jpg
0092_0092.jpg
0804_0804.jpg
0761_0761.jpg
0400_0400.jpg
0228_0228.jpg
0045_0045.jpg
0859_0859.jpg
0776_0776.jpg
0675_0675.jpg
0163_0163.jpg
0930_0930.jpg
0145_0145.jpg
0090_0090.jpg
0448_0448.jpg
0192_0192.jpg
0416_0416.jpg
0531_0531.jpg
0966_0966.jpg
0915_0915.jpg
0598_0598.jpg
0738_0738.jpg
0126_0126.jpg
0567_0567.jpg
0155_0155.jpg
0843_0843.jpg
0662_0662.jpg
0272_0272.jpg
0116_0116.jpg
0122_0122.jpg
0008_0

In [38]:
np.unique(clean_pred)

array([  0, 255])